In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, PillowWriter



In [172]:
# Создаем сферу с помощью параметрических уравнений
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 50)
R = 6400e3/1.3
x = R * np.outer(np.cos(u), np.sin(v))
y = R * np.outer(np.sin(u), np.sin(v))
z = R * np.outer(np.ones(np.size(u)), np.cos(v))
# import Constants
# print(np.linalg.norm(Constants.ECEF) > R)


In [99]:
earth_texture = plt.imread('textures/8k_earth_daymap.jpg') / 255.0

In [5]:
import pandas as pd
from pandas._typing import FilePath


def load_df(name: FilePath, H):
    df = pd.read_csv(
        name,
        sep=';', header=None,
        names=['svId', 'gnssId', 'TOW', 'alm_x', 'alm_y', 'alm_z', 'eph_x', 'eph_y', 'eph_z',
               'elev', 'azim', 'doMes', 'cpMes', 'prMes',
               'alm_x1', 'alm_y1', 'alm_z1', 'eph_x1', 'eph_y1', 'eph_z1'])
    if H:
        df = df[df.TOW % H == 0]
    df = df[df.TOW > 1]

    df.reset_index(drop=True, inplace=True)
    return df

C:\Users\kippe\AppData\Local\Temp\ipykernel_1712\4091001819.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
df = load_df('sat_raw_calc_data.txt', 5*60)  # , time_correction_minus=True)
df = df[df.gnssId == 'GNSS.GPS']
df = df[df.TOW > 72 * 3600]
df = df[df.TOW < 96 * 3600]
df

In [ ]:
from Transformations import eci2ecef

df['eph'] = df.eph_x.notna()
df['visible'] = df.elev.notna() & df.elev > 0

df['alm'] = df.apply(lambda row: (row['alm_x'], row['alm_y'], row['alm_z']), axis=1)
df['alm_ecef'] = df.apply(lambda row: eci2ecef(row['TOW'], *row['alm']), axis=1)
df['X'] = df.alm_ecef.apply(lambda xyz: xyz[0])
df['Y'] = df.alm_ecef.apply(lambda xyz: xyz[1])
df['Z'] = df.alm_ecef.apply(lambda xyz: xyz[2])
# 
# df['X'] = df.alm_x
# df['Y'] = df.alm_y
# df['Z'] = df.alm_z

df = df[['svId', 'TOW', 'X', 'Y', 'Z', 'eph', 'visible']].copy()
df = df.dropna()

In [ ]:
fig, axs = plt.subplots(3, figsize=(10, 8))
for svId in range(1, 33):
    sat = df[df.svId == svId]
    sat = sat[sat.TOW > 10]
    if not len(sat):
        continue
    for i in range(3):
        type = 'XYZ'[i]
        axs[i].plot(sat.TOW, sat[type], label=svId)
plt.savefig('b.png', dpi=600)

In [ ]:
import Constants
from matplotlib.colors import LightSource


# Создаем фигуру и оси
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

TIME = df.TOW.unique().tolist()
print(TIME)
print(len(TIME))
print(len(TIME)/15)
# Отрисовываем сферу с текстурой
surface = ax.plot_surface(x, y, z, rstride=4, cstride=4, linewidth=0, antialiased=False)
# ax.plot_surface(x, y, z, c=earth_texture)

sats = df[df.TOW == TIME[0]]
# global points 
# points = ax.scatter(sats.alm_x, sats.alm_y, sats.alm_z, color='blue')

def get_color(row):
    if row['eph']:
        return 'green'  
    elif row['visible']:
        return'yellow'
    else:
        return 'blue'
print(Constants.ECEF)
# Функция для обновления анимации
def update(time, ax):
    alm = 0
    eph = 0
    visible = 0
    # time = 259500
    ax.cla()
    ax.clear()
    # global points
    sats = df[df.TOW == time]
    # sats = sats[sats.svId == 4]
    # sats = sats[sats.svId >= 25]
    
    ls = LightSource(azdeg=0, altdeg=45)  # Устанавливаем угол направления света
    surface = ax.plot_surface(x, y, z, rstride=4, cstride=4, linewidth=0, antialiased=False, facecolors=ls.shade(z, cmap=plt.cm.cool), zorder=-1, alpha=1)
    
    # ax.plot([Constants.ECEF[0]]*2, [Constants.ECEF[1]]*2, [-2.5e7, 2.5e7], color='red')

    for svId in range(1, 33):
        sat = sats[sats.svId == svId]
        if not len(sat):
            continue
        sat = sat.iloc[0]
        color = get_color(sat)
        ax.scatter(sat.X, sat.Y, sat.Z, color=color, s=15)
        
        if sat.visible:
            ax.plot([Constants.ECEF[0], sat.X], [Constants.ECEF[1], sat.Y], [Constants.ECEF[2], sat.Z], color='yellow', zorder=100, linewidth=1)
        
        d = df[df.svId == svId]
        # if svId > 29:
        ax.plot(d.X, d.Y, d.Z, linewidth=0.5, color='lightgrey', linestyle='--')
        
        if sat.eph:
            eph += 1
        elif sat.visible:
            visible += 1
        else:
            alm += 1
        
    # ax.scatter(sats.alm_x, sats.alm_y, sats.alm_z)
    # np
    # points._offsets3d = ([], [], [])

    # colors = sats.apply(get_color, axis=1).tolist()
    
    # points = ax.scatter(
    #     sats.X, sats.Y, sats.Z,
    #     color=colors
    # )
    for i in range(3):
        ax.scatter(0, 0, 0, color=['blue', 'green', 'yellow'][i], s=10, label=[f'alm [{alm}]', f'eph [{eph}]', f'visible [{visible}]'][i])
    ax.legend()
    
    ax.set_xlim3d(-2.5e7, 2.5e7)
    ax.set_ylim3d(-2.5e7, 2.5e7)
    ax.set_zlim3d(-2.5e7, 2.5e7)
    
    ax.scatter(*Constants.ECEF, color='red', s=100, zorder=2000)

    # points._offsets3d = (sats.alm_x, sats.alm_y, sats.alm_z)
    # colors = sats.apply(get_color, axis=1)
    # points.set_color(colors.tolist())
    # print(time, colors.tolist())
    
    ax.view_init(azim=60)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.text2D(0.1, 0.9, 'Скорость: x3600', transform=ax.transAxes, fontsize=12,
          verticalalignment='bottom', horizontalalignment='center')
    
    
update(TIME[0], ax)
# update(239100, ax)
# Создаем анимацию
ani = FuncAnimation(fig, update, frames=TIME, fargs=(ax, ), interval=1)#np.arange(0, 3600*24, 300)
# ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 10), fargs=(ax, ), interval=1)#np.arange(0, 3600*24, 300)

# Создаем объект writer для сохранения анимации в формате GIF
writer = PillowWriter(fps=12)
# plt.annotate('Скорость: x3600', xy=(0.5, 0), xytext=(0, 20),
#              xycoords='axes fraction', textcoords='offset points',
#              ha='center', va='bottom', fontsize=12)

# Сохраняем анимацию в файл
ani.save('animation.gif', writer=writer, dpi=300)#, bbox_inches='tight')
# ani.save('animation.gif', writer='imagemagik', dpi=300, fps=12)#transparent=True)
# plt.savefig('c.png', dpi=300, transparent=False)
plt.show()



In [139]:
# Сохраняем анимацию в файл
ani.save('animation2.gif', writer=writer, dpi=600)
# plt.savefig('a.png', dpi=600)
# plt.show()